- 교제 정보 : https://goo.gl/eFk5uN

# Lesson1. Extended Relational Algebra

----

## Topic1. Extended Relational Algebra Operation

### 1-1. Aggregation: GROUP BY

영역별로 그룹을 지어서 그 중에서 계산된 값을 반환한다.
$$ _{Branch}G_{max(Balance)}(Bank) $$

- 예 : Bank라는 테이블을 만든뒤 각 브랜치별로 그룹을 지어서 그 중의 최대 잔고를 조회하는 예
- 식 : **SELECT Branch, max(Balance) FROM Bank GROUP BY Branch**

실습 참고 
- SQLite in Jupyter Notebook : https://docs.python.org/2/library/sqlite3.html
- Install SQLite in Mac : https://goo.gl/ruiZEK

In [33]:
import sqlite3 as sql

In [71]:
# 작성한 테이블 삭제용
conn = sql.connect("Bank.sqlite")
cursor = conn.cursor()
cursor.execute("""DROP TABLE Bank""")

In [72]:
conn = sql.connect("Bank.sqlite")
cursor = conn.cursor()
cursor.execute("""CREATE TABLE Bank(Account int, Branch text, Country text, Balance text) """)
cursor.execute("INSERT INTO Bank(Account, Branch, Country, Balance) VALUES(100090, 'Seoul', 'Korea', 10000)")
cursor.execute("INSERT INTO Bank(Account, Branch, Country, Balance) VALUES(100100, 'Busan', 'Korea', 9000)")
cursor.execute("INSERT INTO Bank(Account, Branch, Country, Balance) VALUES(200010, 'New York', 'USA', 20000)")
cursor.execute("INSERT INTO Bank(Account, Branch, Country, Balance) VALUES(100090, 'Seoul', 'Korea', 3000)")
cursor.execute("INSERT INTO Bank(Account, Branch, Country, Balance) VALUES(100100, 'Busan', 'Korea', 4000)")
cursor.execute("INSERT INTO Bank(Account, Branch, Country, Balance) VALUES(200010, 'New York', 'USA', 15000)")
conn.commit()

In [73]:
conn = sql.connect("Bank.sqlite")
cursor = conn.cursor()
for row in cursor.execute("SELECT Branch, max(Balance) FROM Bank GROUP BY Branch"):
    print(row)
cursor.close()

('Busan', '9000')
('New York', '20000')
('Seoul', '3000')


### 1-2. Aggregation: GROUP BY ... HAVING ....

그룹별로 정렬한 것 중에서 조건식을 적용할 수 있다.

$$ \delta_{sum(Balance)>10,000}(_{Branch}G_{Balance}(Bank)) $$

- 예: 각 브랜치별로 정렬한 후 최고 잔액을 구하여, 그 중 10,0000 이상인 것만 조회

----